In [6]:
import cv2
import os
import sys
import numpy as np
import imutils
import math
print(os.getcwd())

C:\Users\MUJ\Desktop\Gait-Recognition-post-testing


In [2]:
def getFeatures(sample_image,sample_image_rgb):
    thresh = cv2.threshold(sample_image, 45, 255, cv2.THRESH_BINARY)[1]

    #Find the ccontour of the binary image
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)#just graps the countours from above received cnts that also contains other info 
    c = max(cnts, key=cv2.contourArea)

    #The extreme most points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    """
        In the below code :
            1) We are considering the points only below fist level that is coordinates with y value greater than that
                of fist
            2) In those values we are picking the rightmost and leftmost point that eventually comes out to
                be the foot points
        
    """
    maximum_y_r = -999
    maximum_x_r = -999
    maximum_x_l = extLeft[0]
    maximum_y_l = -999 
    
    for i in cnts[0]:
        for k in i:
            if k[1] > 150:
                if maximum_x_r < k[0]:
                    maximum_x_r = k[0]
                    maximum_y_r = k[1]
                        
    maximum_x_l = maximum_x_r
    maximum_y_l = -999 
    for i in cnts[0]:
        for k in i:
            if k[1] > maximum_y_r - 10:
                if maximum_x_l > k[0]:
                    maximum_x_l = k[0]
                    maximum_y_l = k[1]
                    
    if maximum_y_l > maximum_y_r:
        b=maximum_y_l
    else:
        b=maximum_y_r
    dx2 = (maximum_x_r-extBot[0])**2          
    dy2 = (maximum_y_r-extBot[1])**2          
    distance = math.sqrt(dx2 + dy2)
    hgt = b-extTop[1]
    w=extRight[0]-extLeft[0]
    
    return distance,hgt,w

In [3]:
def getCroppedImage(sample_image,sample_image_rgb,width_list_max,height_list_max):
    thresh = cv2.threshold(sample_image, 45, 255, cv2.THRESH_BINARY)[1]
    
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    x=extLeft[0]
    y=extTop[1]
    
    mid_point = x + width_list_max//2
    if (extTop[0] < mid_point):
        diff = mid_point - extTop[0]
        x = x - diff
    elif extTop[0] > mid_point:
        diff = extTop[0] - mid_point
        x += diff
    if x < 0:
        x = 0
    elif x > 240:
        x = 240
    cropped = sample_image[y:y+height_list_max,x:x+width_list_max] # [y_min:y_max,x_min:x_max]
    return cropped

In [ ]:
sample_image_path = "C:\\Users\\MUJ\\Desktop\\minor\\casia b\\silhouettes\\silhouette\\001\\nm-01\\090\\001-nm-01-090-"
sample_image_rgb_path = "C:\\Users\\MUJ\\Desktop\\minor\\casia b\\silhouettes\\silhouette\\001\\nm-01\\090\\001-nm-01-090-"

distance_list = []
height_list = []
width_list=[]

"""
    Extract contours and features of the subject like:
        1) Height
        2) Stride Length

"""
for i in range(47,103):
    sample_image = cv2.imread(sample_image_path + "{0:03}".format(i) +".png",0)
    sample_image_rgb = cv2.imread(sample_image_rgb_path + "{0:03}".format(i) +".png") 
    distance,height,width = getFeatures(sample_image,sample_image_rgb)
    distance_list.append(round(distance))
    if height>=120:
        height_list.append(height)
    width_list.append(width)   

"""
    Gets the cropped image from given above features

"""

width_list_max = max(width_list)
height_list_max = max(height_list)
image_list = []

for i in range(47,103):
    sample_image = cv2.imread(sample_image_path + "{0:03}".format(i) +".png",0)
    sample_image_rgb = cv2.imread(sample_image_rgb_path + "{0:03}".format(i) +".png") 
    cropped_image = getCroppedImage(sample_image,sample_image_rgb,width_list_max,height_list_max)
    print(cropped_image.shape)
    image_list.append(cropped_image)
    if i % 10 == 0:
        gei_image = np.zeros(cropped_image.shape,dtype = np.int)
        gei_image = np.mean(image_list,axis=0)
        gei_image = gei_image.astype(np.int)
        cv2.imwrite("gei{}.jpg".format(i),gei_image)
        image_list = []